In [2]:
import tensorflow as tf
from tensorflow import keras
import kerastuner
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import os
import json
import datetime

plt.rcParams["figure.figsize"] = (20, 5)

physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], enable=True)

In [3]:
dataset_name = "SEG_2"

In [4]:
dataset = np.genfromtxt("data/{}_train_set.csv".format(dataset_name), delimiter="\n", dtype=np.float32) #np.int64
dataset

array([0., 0., 0., ..., 1., 3., 1.], dtype=float32)

In [5]:
word_index = np.genfromtxt("data/word_index.csv", delimiter="\n", dtype=np.int64)
vocab_size = len(word_index)
vocab_size

14882

In [66]:
param_list = dict()

param_list["PAST_HISTORY"] = 16
param_list["FUTURE_TARGET"] = 8
param_list["BATCH_SIZE"] = 128
param_list["EPOCHS"] = 100
param_list["BUFFER_SIZE"] = 200000
param_list["VOCAB_SIZE"] = vocab_size
param_list["LEARNING_RATE"] = 0.01
param_list["NUM_1_NEURONS"] = 177
param_list["NUM_2_NEURONS"] = 177
param_list["DROPOUT_1"] = 0.1
param_list["DROPOUT_2"] = 0.2

In [67]:
def generate_timeseries(dataset, start_index, end_index, history_size, target_size):
    data = []
    labels = []

    start_index = start_index + history_size
    if end_index is None:
        end_index = len(dataset) - target_size

    for i in range(start_index, end_index):
        indices = range(i-history_size, i)
        # Reshape data from (history_size,) to (history_size, n_feature)
        data.append(np.reshape(dataset[indices], (history_size, 1)))
        #data.append(dataset[indices])
        labels.append(np.reshape(dataset[i:i+target_size], (target_size, 1)))
        #labels.append(dataset[i:i+target_size])
    return np.array(data), np.array(labels)

In [68]:
x_train, y_train = generate_timeseries(dataset, 0, None, param_list["PAST_HISTORY"], param_list["FUTURE_TARGET"])
x_train.shape, y_train.shape

((14858, 16, 1), (14858, 8, 1))

In [69]:
from tensorflow.keras.layers import TimeDistributed, Dense, Conv1D, MaxPooling1D, Bidirectional, LSTM, Flatten, Reshape, Lambda

In [93]:
model = keras.models.Sequential()
model.add(Conv1D(filters=64, kernel_size=9, padding="causal", activation='relu'))
#model.add(Conv1D(filters=32, kernel_size=3, padding="causal", activation='relu'))
model.add(MaxPooling1D(pool_size=2))
#model.add(Flatten())
#model.add(Lambda(lambda x:tf.expand_dims(x,axis=-1),input_shape=[None]))

model.add(LSTM(64, return_sequences=True))
model.add(LSTM(64, return_sequences=True))
#model.add(LSTM(32, return_sequences=True))
model.add(TimeDistributed(Dense(param_list["VOCAB_SIZE"], activation="softmax")))

model.compile(optimizer=keras.optimizers.Nadam(), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [94]:
result = model.predict(x_train[10000].reshape(1, -1, 1))
result.shape

(1, 8, 14882)

In [95]:
model_history = model.fit(x_train, y_train, batch_size=param_list["BATCH_SIZE"], validation_split=0.2, epochs=param_list["EPOCHS"])

0s - loss: 0.5676 - accuracy: 0.8683/93 [=========================>....] - ETA: 0s - loss: 0.5674 - accuracy: 0.8685/93 [==========================>...] - ETA: 0s - loss: 0.5662 - accuracy: 0.8687/93 [===========================>..] - ETA: 0s - loss: 0.5671 - accuracy: 0.8689/93 [===========================>..] - ETA: 0s - loss: 0.5683 - accuracy: 0.8691/93 [============================>.] - ETA: 0s - loss: 0.5669 - accuracy: 0.8693/93 [==============================] - 3s 32ms/step - loss: 0.5675 - accuracy: 0.8668 - val_loss: 0.7037 - val_accuracy: 0.7345
Epoch 96/100
 1/93 [..............................] - ETA: 0s - loss: 0.3945 - accuracy: 0.88 3/93 [..............................] - ETA: 1s - loss: 0.6135 - accuracy: 0.83 5/93 [>.............................] - ETA: 1s - loss: 0.6298 - accuracy: 0.84 7/93 [=>............................] - ETA: 1s - loss: 0.6117 - accuracy: 0.85 9/93 [=>............................] - ETA: 1s - loss: 0.5993 - accuracy: 0.8511/93 [==>.............

In [96]:
timestamp = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
'''
log_dir = "logs/fit/" + timestamp
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
'''
version_dir = "version/" + timestamp 

os.makedirs(version_dir)
timestamp

'20200909-155423'

In [97]:
model.save("version/{}/model.h5".format(timestamp))